In [1]:
"""
Hauptprogramm für die Energiespeicher-Simulation.
Koordiniert den Ablauf der Simulation und die Auswertung der Ergebnisse.
"""
import os
import time
import pandas as pd
pd.set_option('compute.use_numba', True)
pd.set_option('compute.use_numexpr', True)
import numpy as np
import matplotlib.pyplot as plt
import argparse
from datetime import datetime, timedelta

# Module importieren
from utils.logging_setup import setup_logging
from utils.localization import setup_german_locale, format_date_german, get_german_weekday
from data.data_loader import load_price_data
from models.energy_storage import EnergyStorage
from strategies.threshold_lookahead import threshold_lookahead
from visualization.plotting import visualize_day
from economic_analysis.analyzer import run_economic_analysis
# Alle Plotting-Funktionen in einem Import
from visualization.plotting import (
    plot_cycle_analysis,
    plot_charge_discharge_patterns,
    plot_efficiency_analysis,
    plot_price_arbitrage_analysis,
)

# Konfiguration importieren
import config


# --- STIL-KONFIGURATION ---

# Lade den Seaborn-Stil.
plt.style.use('seaborn-v0_8-whitegrid')

# Überschreibe und verfeinere die Details mit deinen rcParams.
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['Calibri'],  

# --- Deine gewünschten Schriftgrößen ---
    'figure.titlesize': 22,
    'axes.titlesize': 20,
    'axes.labelsize': 18,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 16,

# --- Deine weiteren Anpassungen ---
    'axes.titlepad': 20,
    'lines.linewidth': 2.5, 
    'lines.markersize': 8
})


def save_config_to_output(output_dir):
    """
    Speichert die aktuelle Konfiguration in das Ausgabeverzeichnis für Reproduzierbarkeit.
    """
    import inspect
    import config
    
    config_file_path = os.path.join(output_dir, "config_snapshot.py")
    
    with open(config_file_path, 'w', encoding='utf-8') as f:
        f.write("# Konfiguration zum Zeitpunkt der Simulation\n")
        f.write("# Automatisch generiert von der Simulationssoftware\n\n")
        
        # Alle Attribute aus config auslesen, die keine Funktionen sind
        for name, value in inspect.getmembers(config):
            if not name.startswith('__') and not inspect.isfunction(value) and not inspect.ismodule(value):
                if isinstance(value, str):
                    f.write(f"{name} = '{value}'\n")
                else:
                    f.write(f"{name} = {value}\n")


def run_continuous_simulation(prices_df, output_dir, save_plots=True, show_plots=False):
    """
    Führt eine kontinuierliche Simulation über den gesamten Datensatz durch.
    
    Parameters:
    -----------
    prices_df : DataFrame
        DataFrame mit den Preisdaten
    output_dir : str
        Verzeichnis für Ausgabedateien
    save_plots : bool, optional
        Ob Plots gespeichert werden sollen
    show_plots : bool, optional
        Ob Plots angezeigt werden sollen
    
    Returns:
    --------
    tuple
        (daily_results, storage, total_profit, annual_cycles)
    """
    logger = setup_logging()
    logger.info("\n=== Starte optimierte kontinuierliche Simulation über den gesamten Datensatz ===")
    
    # Initialisiere Speicher und führe die DP-Strategie aus
    storage = EnergyStorage(
        capacity_mwh=config.CAPACITY_MWH,
        charge_rate=config.CHARGE_DISCHARGE_RATE,
        efficiency=config.EFFICIENCY,
        fee_per_mwh=config.FEE_PER_MWH
    )
    
    logger.info(f"🔋 Initialer Speicherstand: {storage.energy_level:.2f} MWh (0.0%)")
    
    # DP-Strategie auf den gesamten Datensatz anwenden
    logger.info("Wende TH Strategie auf den gesamten Datensatz an...")
    logger.info(f"Dies kann einige Zeit dauern für {len(prices_df)} Zeitpunkte...")
    
    start_time = time.time()
    storage, energy_history = threshold_lookahead(
        prices_df, 
        storage, 
        window_size=config.ROLLING_WINDOW_SIZE
    )
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    logger.info(f"Optimierung abgeschlossen in {elapsed_time:.2f} Sekunden")
    
    # Zusammenfassung der globalen Simulation
    total_transactions = len(storage.transactions)
    total_charged = sum([t.get('amount', 0) for t in storage.transactions if t['type'] == 'charge'])
    total_discharged = sum([t.get('amount_gross', t.get('amount', 0)) for t in storage.transactions if t['type'] == 'discharge'])
    total_profit = storage.cash
    
    logger.info("\n=== Ergebnis der kontinuierlichen Simulation ===")
    logger.info(f"Gesamtzahl Transaktionen: {total_transactions}")
    logger.info(f"Gesamte geladene Energie: {total_charged:.2f} MWh")
    logger.info(f"Gesamte entladene Energie: {total_discharged:.2f} MWh")
    logger.info(f"Gesamtzyklen: {storage.total_cycles:.2f}")
    logger.info(f"Gesamtgewinn: {total_profit:.2f} €")
    logger.info(f"Finaler Speicherstand: {storage.energy_level:.2f} MWh ({storage.energy_level/storage.capacity*100:.1f}%)")
    
    # Verzeichnis für tägliche Visualisierungen
    daily_viz_dir = os.path.join(output_dir, "handelsentscheidungen")
    os.makedirs(daily_viz_dir, exist_ok=True)
    
    # Vorverarbeitung der Daten nach Datum
    logger.info("\n=== Vorverarbeitung der Daten nach Datum ===")
    
    # Transaktionen nach Datum gruppieren
    logger.info("Gruppiere Transaktionen nach Datum...")
    transactions_by_date = {}
    for t in storage.transactions:
        if t['index'] < len(prices_df):
            tx_time = prices_df.iloc[t['index']]['datetime']
            date = tx_time.date()
            if date not in transactions_by_date:
                transactions_by_date[date] = []
            transactions_by_date[date].append(t.copy())
    
    # Energiehistorie nach Datum gruppieren
    logger.info("Gruppiere Energiehistorie nach Datum...")
    energy_by_date = {}
    for eh in energy_history:
        if eh['time_index'] < len(prices_df):
            eh_time = prices_df.iloc[eh['time_index']]['datetime']
            date = eh_time.date()
            if date not in energy_by_date:
                energy_by_date[date] = []
            energy_by_date[date].append(eh.copy())
    
    # Mapping zwischen globalen und täglichen Indizes für jeden Tag erstellen
    index_mapping_by_date = {}
    logger.info("Erstelle Index-Mappings für jeden Tag...")
    
    for date, date_dt in ((d, pd.to_datetime(d)) for d in sorted(set(prices_df['datetime'].dt.date))):
        day_prices = prices_df[prices_df['datetime'].dt.date == date].reset_index()
        
        # Mapping zwischen globalem und täglichem Index erstellen
        global_to_day_index = {}
        for i, row in day_prices.iterrows():
            if 'index' in row:
                global_to_day_index[row['index']] = i
        
        index_mapping_by_date[date] = global_to_day_index
    
    # Ergebnisse pro Tag auswerten mit optimierten Datenzugriff
    all_dates = sorted(prices_df['datetime'].dt.date.unique())
    daily_results = []
    
    logger.info("\n=== Erstelle tägliche Ergebnisse ===")
    total_days = len(all_dates)
    start_time = time.time()  # Zeit messen
    
    for day_index, date in enumerate(all_dates):
        # Fortschritt anzeigen
        if day_index % 20 == 0:
            progress = (day_index + 1) / total_days * 100
            logger.info(f"[{progress:.1f}%] Erstelle Ergebnis für Tag {day_index+1}/{total_days}: {format_date_german(pd.to_datetime(date))}")
        
        # Datum in datetime konvertieren
        date_dt = pd.to_datetime(date)
        
        # Direkter Zugriff auf die vorgruppierte Daten
        day_transactions = transactions_by_date.get(date, [])
        day_transactions_df = pd.DataFrame(day_transactions) if day_transactions else pd.DataFrame()
        
        # Filtern der Preisdaten für diesen Tag
        day_prices = prices_df[prices_df['datetime'].dt.date == date].copy().reset_index()
        
        # Verwende das vorberechnete Mapping
        global_to_day_index = index_mapping_by_date.get(date, {})
        
        # Transaktionen-Indizes anpassen (falls vorhanden)
        if not day_transactions_df.empty and 'index' in day_transactions_df.columns:
            # Original-Index speichern für Referenz
            day_transactions_df['original_index'] = day_transactions_df['index'].copy()
            
            # Indizes umwandeln von global zu tagesspezifisch
            day_transactions_df['index'] = day_transactions_df['index'].apply(
                lambda x: global_to_day_index.get(x, 0) if x in global_to_day_index else 0
            )
        
        # Direkter Zugriff auf die vorgruppierte Energiehistorie
        day_energy_history = energy_by_date.get(date, [])
        
        # Tagesenergie-Indizes anpassen
        for eh in day_energy_history:
            if eh['time_index'] in global_to_day_index:
                eh['time_index'] = global_to_day_index[eh['time_index']]
            else:
                # Fallback falls der Index nicht im Mapping ist
                eh['time_index'] = 0
        
        # Tägliche Statistiken berechnen
        if not day_transactions_df.empty:
            # Berechne Tagesgewinn
            day_profit = 0
            for t in day_transactions:
                if t['type'] == 'discharge':
                    day_profit += t.get('revenue', 0)
                else:
                    day_profit -= t.get('cost', 0)
            
            # Berechne Tageszyklen aus Entladungen
            day_discharge = sum([t.get('amount_gross', t.get('amount', 0)) 
                               for t in day_transactions if t['type'] == 'discharge'])
            day_cycles = day_discharge / config.CAPACITY_MWH
            
            # Bestimme Anfangs- und Endladestand des Tages
            if day_energy_history:
                initial_energy = day_energy_history[0]['energy_level']
                final_energy = day_energy_history[-1]['energy_level']
            else:
                initial_energy = 0
                final_energy = 0
            
            # Anzahl der Handelsgeschäfte
            charge_count = len([t for t in day_transactions if t['type'] == 'charge'])
            discharge_count = len([t for t in day_transactions if t['type'] == 'discharge'])
            
            # Tagesergebnis speichern
            daily_result = {
                'date': date_dt,
                'profit': day_profit,
                'cycles_completed': day_cycles,
                'charge_count': charge_count,
                'discharge_count': discharge_count,
                'initial_energy_level': initial_energy,
                'final_energy_level': final_energy,
                'transactions': day_transactions_df,
                'prices_df': day_prices,
                'energy_history': day_energy_history
            }
            
            # Gesamtstatistiken
            day_charged = sum([t.get('amount', 0) for t in day_transactions if t['type'] == 'charge'])
            day_discharged_gross = sum([t.get('amount_gross', t.get('amount', 0)) 
                                     for t in day_transactions if t['type'] == 'discharge'])
            day_discharged_net = sum([t.get('amount_usable', t.get('amount', 0) * config.EFFICIENCY) 
                                   for t in day_transactions if t['type'] == 'discharge'])
            
            # Erweiterte Statistiken
            daily_result.update({
                'total_charged': day_charged,
                'total_gross_energy': day_discharged_gross,
                'total_usable_energy': day_discharged_net,
                'total_energy_loss': day_discharged_gross - day_discharged_net,
                'total_fees': sum([t.get('transaction_fee', 0) for t in day_transactions]),
                'efficiency_losses': sum([t.get('efficiency_loss', 0) for t in day_transactions if t['type'] == 'discharge'])
            })
            
            daily_results.append(daily_result)
        else:
            # Leeres Tagesergebnis für Tage ohne Transaktionen
            daily_results.append({
                'date': date_dt,
                'profit': 0,
                'cycles_completed': 0,
                'charge_count': 0,
                'discharge_count': 0,
                'initial_energy_level': 0 if not day_energy_history else day_energy_history[0]['energy_level'],
                'final_energy_level': 0 if not day_energy_history else day_energy_history[-1]['energy_level'],
                'transactions': pd.DataFrame(),
                'prices_df': day_prices,
                'energy_history': day_energy_history
            })
    
    # Zeitmessung für die Tagesberechnung
    end_time = time.time()
    elapsed_time = end_time - start_time
    logger.info(f"Tagesberechnung abgeschlossen in {elapsed_time:.2f} Sekunden")
    
    # Phase 2: Parallele Tagesvisualisierungen erstellen
    logger.info("\n=== Starte parallele Tagesvisualisierung ===")
    from visualization.parallel_plotting import create_visualizations_parallel
    
    create_visualizations_parallel(
        daily_results,
        prices_df,
        output_dir=daily_viz_dir,
        save_plots=save_plots,
        show_plots=False,  # Immer False für parallele Verarbeitung
        visualization_frequency=config.VISUALIZE_FREQUENCY
    )
    
    # Berechnung für jährliche Hochrechnung
    days_in_data = len(all_dates)
    total_profit = sum(result['profit'] for result in daily_results)
    total_cycles = sum(result['cycles_completed'] for result in daily_results)
    
    annual_profit = total_profit * (365 / days_in_data) if days_in_data < 365 else total_profit
    annual_cycles = total_cycles * (365 / days_in_data) if days_in_data < 365 else total_cycles
    
    logger.info(f"\n=== Gesamtergebnis der täglichen Auswertungen ===")
    logger.info(f"Gesamtgewinn aus {days_in_data} Tagen: {total_profit:.2f} €")
    logger.info(f"Extrapolierter Jahresgewinn: {annual_profit:.2f} €")
    logger.info(f"Gesamtzyklen: {total_cycles:.2f}")
    logger.info(f"Durchschnittliche Zyklen pro Tag: {total_cycles/days_in_data:.2f}")
    logger.info(f"Extrapolierte jährliche Zyklen: {annual_cycles:.2f}")
    
    return daily_results, storage, annual_profit, annual_cycles


def main():
    """
    Hauptfunktion der Energiespeicher-Simulation.
    """
    # Logger setup
    logger = setup_logging()
    logger.info("Starte Energiespeicher-Simulation")
    
    # Deutsche Lokalisierung einrichten
    if setup_german_locale():
        logger.info("Deutsche Lokalisierung erfolgreich eingerichtet")
    else:
        logger.warning("Deutsche Lokalisierung nicht verfügbar, verwende interne Übersetzungstabellen")
    
    # Kommandozeilenargumente parsen
    parser = argparse.ArgumentParser(description='Energiespeicher-Simulation')
    parser.add_argument('--file', '-f', type=str, default=config.DEFAULT_PRICE_FILE,
                        help='Pfad zur CSV-Datei mit Preisdaten')
    parser.add_argument('--output', '-o', type=str, default=config.OUTPUT_DIR,
                        help='Basis-Verzeichnis für die Ausgabedateien')
    parser.add_argument('--visualize', '-v', action='store_true',
                        help='Plots anzeigen (standardmäßig nur speichern)')
    
    try:
        args = parser.parse_args()
    except:
        # Fallback für Jupyter Notebook
        class Args:
            def __init__(self):
                self.file = config.DEFAULT_PRICE_FILE
                self.output = config.OUTPUT_DIR
                self.visualize = False
        args = Args()
    
    # Startzeit für Laufzeitmessung
    start_time = time.time()
    
    # Zeitstempel für eindeutiges Ausgabeverzeichnis erstellen
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")    
    base_output_dir = args.output
    output_dir = os.path.join(base_output_dir, f"simulation_{timestamp}")
    
    logger.info(f"Erstelle neues Ausgabeverzeichnis mit Zeitstempel: {output_dir}")
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # CSV-Datei einlesen
    logger.info(f"Inhalt von config.DEFAULT_PRICE_FILE: {config.DEFAULT_PRICE_FILE}")
    prices_df = load_price_data(config.DEFAULT_PRICE_FILE)
    logger.info(f"Verwende direkt den Pfad aus config: {config.DEFAULT_PRICE_FILE}")
    if prices_df is None:
        logger.error("Fehler beim Laden der Daten. Bitte Dateipfad überprüfen.")
        return
    
    logger.info(f"\n=== Optimierte Analyse mit fester Gebühr: {config.FEE_PER_MWH} € pro MWh ===")
    
    # Kontinuierliche Simulation über den gesamten Datensatz
    daily_results, storage, annual_profit, annual_cycles = run_continuous_simulation(
        prices_df, 
        output_dir=output_dir,
        save_plots=config.SAVE_PLOTS,
        show_plots=args.visualize
    )

    # Unterverzeichnis für wirtschaftliche Analyse erstellen
    econ_dir = os.path.join(output_dir, "wirtschaftliche_analyse")
    os.makedirs(econ_dir, exist_ok=True)
    
    # Wirtschaftliche Analyse mit dem tatsächlichen Gewinn und Zyklen
    try:
        # Importiere die notwendigen Funktionen für wirtschaftliche Analyse
        from economic_analysis.analyzer import run_economic_analysis
        from visualization.plotting import (
            plot_economic_results_extended,
            plot_breakeven_scenarios,
            plot_monthly_profit_analysis,
            plot_trading_heatmap
        )
        
        # Führe wirtschaftliche Analyse durch
        economic_results = run_economic_analysis(
            prices_df, 
            output_dir=output_dir,
            annual_profit=annual_profit,
            annual_cycles=annual_cycles,
            daily_results=daily_results
        )
        
        # Explizit alle wirtschaftlichen Plots erstellen
        if economic_results:
            logger.info("Erstelle erweiterte wirtschaftliche Visualisierungen...")
            
            # Pfad für die Plots
            plot_path = os.path.join(econ_dir, "economic_analysis")
            
            # Erweiterte wirtschaftliche Ergebnisse plotten
            try:
                detailed_results = plot_economic_results_extended(economic_results, plot_path)
                if detailed_results and 'breakeven_point' in detailed_results:
                    if detailed_results['breakeven_point']:
                        logger.info(f"Break-Even (nominal): {detailed_results['breakeven_point']:.2f} Jahre")
                    else:
                        logger.info(f"Break-Even (nominal): > {config.SIMULATION_YEARS} Jahre")
            except Exception as e:
                logger.warning(f"Fehler bei den erweiterten wirtschaftlichen Ergebnissen: {e}")
            
            # Break-Even-Szenarien plotten
            try:
                breakeven_results = plot_breakeven_scenarios(economic_results, plot_path)
                if breakeven_results and 'standard_breakeven' in breakeven_results:
                    be_years = breakeven_results['standard_breakeven']
                    if be_years:
                        if be_years < config.SIMULATION_YEARS:
                            logger.info(f"Break-Even im Standardszenario: {be_years:.2f} Jahre")
                        else:
                            logger.info(f"Break-Even im Standardszenario: > {config.SIMULATION_YEARS} Jahre")
                    else:
                        logger.info(f"Break-Even im Standardszenario: > {config.SIMULATION_YEARS} Jahre")
            except Exception as e:
                logger.warning(f"Fehler bei den Break-Even-Szenarien: {e}")
            
            # Monatliche Gewinnanalyse plotten
            try:
                monthly_results = plot_monthly_profit_analysis(daily_results, plot_path)
                if monthly_results:
                    logger.info(f"Monatliche Gewinnanalyse erstellt: Bester Monat ist {monthly_results.get('best_month', 'N/A')}")
            except Exception as e:
                logger.warning(f"Fehler bei der monatlichen Gewinnanalyse: {e}")
            
            # Trading-Heatmap plotten
            try:
                trading_results = plot_trading_heatmap(daily_results, plot_path)
                if trading_results:
                    charge_hour = trading_results.get('most_active_charge_hour')
                    if charge_hour is not None:
                        logger.info(f"Trading-Heatmap erstellt: Beste Ladestunde ist {charge_hour}:00 Uhr")
                    else:
                        logger.info("Trading-Heatmap erstellt: Keine eindeutige beste Ladestunde identifiziert")
            except Exception as e:
                logger.warning(f"Fehler bei der Trading-Heatmap: {e}")
            
            logger.info(f"Wirtschaftliche Analyse-Grafiken gespeichert im Verzeichnis: {econ_dir}")
        else:
            logger.warning("Keine wirtschaftlichen Ergebnisse für Visualisierungen verfügbar")
    
    except Exception as e:
        logger.error(f"Fehler bei der wirtschaftlichen Analyse: {e}")
    
    # Batterie-Parameter-Visualisierungen
    try:
        # Batterie-Parameter-Visualisierungen in separatem Unterordner
        battery_dir = os.path.join(output_dir, "batterie_parameter")
        os.makedirs(battery_dir, exist_ok=True)
        
        # Generischer Pfad für die Batterie-Parameter-Grafiken
        battery_path = os.path.join(battery_dir, "battery_analysis")
        
        # Zyklenanalyse
        try:
            cycle_results = plot_cycle_analysis(daily_results, battery_path)
            logger.info(f"Zyklenanalyse erstellt: Gesamt {cycle_results.get('total_cycles', 0):.1f} Zyklen")
        except Exception as e:
            logger.warning(f"Fehler bei der Zyklenanalyse: {str(e)}")
        
        # Lade-/Entlademuster
        try:
            pattern_results = plot_charge_discharge_patterns(daily_results, battery_path)
            logger.info(f"Lade-/Entlademuster-Analyse erstellt")
        except Exception as e:
            logger.warning(f"Fehler bei der Lade-/Entlademuster-Analyse: {str(e)}")
        
        # Effizienzanalyse
        try:
            efficiency_results = plot_efficiency_analysis(daily_results, battery_path)
            logger.info(f"Effizienzanalyse erstellt: Ø Kapazitätsnutzung {efficiency_results.get('avg_capacity_utilization', 0):.2f}%")
        except Exception as e:
            logger.warning(f"Fehler bei der Effizienzanalyse: {str(e)}")
        
        # Preis-Arbitrage
        try:
            arbitrage_results = plot_price_arbitrage_analysis(daily_results, battery_path)
            logger.info(f"Preis-Arbitrage-Analyse erstellt: Ø Handelsmarge {arbitrage_results.get('avg_margin', 0):.2f} €/MWh")
        except Exception as e:
            logger.warning(f"Fehler bei der Preis-Arbitrage-Analyse: {str(e)}")
        
        logger.info(f"Batterie-Parameter-Grafiken gespeichert im Verzeichnis: {battery_dir}")
        
    except Exception as e:
        logger.warning(f"Fehler bei der Erstellung der Batterie-Parameter-Grafiken: {str(e)}") 
    
    # Laufzeit berechnen und ausgeben
    end_time = time.time()
    elapsed_time = end_time - start_time
    hours, remainder = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    logger.info(f"Gesamtlaufzeit: {int(hours):02d}:{int(minutes):02d}:{seconds:.2f}")
    
    # Speichern der verwendeten Konfiguration für Reproduzierbarkeit
    try:
        save_config_to_output(output_dir)
        logger.info(f"Konfiguration für Reproduzierbarkeit gespeichert in: {output_dir}/config_snapshot.py")
    except Exception as e:
        logger.warning(f"Fehler beim Speichern der Konfiguration: {str(e)}")
    
    logger.info(f"Alle Simulationsergebnisse wurden in {output_dir} gespeichert.")
    
    return output_dir  


if __name__ == "__main__":
    main()

2025-08-09 14:25:36,417 - INFO - Starte Energiespeicher-Simulation
2025-08-09 14:25:36,439 - INFO - Deutsche Lokalisierung erfolgreich eingerichtet
2025-08-09 14:25:36,440 - INFO - Erstelle neues Ausgabeverzeichnis mit Zeitstempel: output_plots\simulation_20250809_142536
2025-08-09 14:25:36,442 - INFO - Inhalt von config.DEFAULT_PRICE_FILE: c:\Users\tillw\OneDrive\Bachelorarbeit\01_Pythoncode\energy-storage-arbitrage-simulator\SMARD_15min_Jahr 2024_DE\Gro_handelspreise_202401010000_202501010000_Viertelstunde.csv
2025-08-09 14:25:36,443 - INFO - Starte Einlesen der Datei: c:\Users\tillw\OneDrive\Bachelorarbeit\01_Pythoncode\energy-storage-arbitrage-simulator\SMARD_15min_Jahr 2024_DE\Gro_handelspreise_202401010000_202501010000_Viertelstunde.csv
2025-08-09 14:25:36,478 - INFO - Erfolgreich eingelesen: 35136 Zeilen
2025-08-09 14:25:36,620 - INFO - Zeitreihe konvertiert: 2024-01-01 00:00:00 bis 2024-12-31 23:45:00
2025-08-09 14:25:36,627 - INFO - 
Aufbereitete Daten:
2025-08-09 14:25:36,627